In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import matplotlib.pyplot as plt
print(tf.__version__)

In [ ]:
from tensorflow.keras.layers import Conv2D, ReLU, MaxPooling2D, Dense, BatchNormalization, Softmax, GlobalAveragePooling2D

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import optimizers

In [ ]:
learning_rate = 0.0002
img_size = 150
batch_size = 16
training_epochs = 50

In [ ]:
n_train = 7488
n_val = 2496
n_class = 12

In [ ]:
cd '/content/gdrive/My Drive/TensorFlow_Training_13th'

In [ ]:
cur_dir = os.getcwd()

tfrecord_dir = os.path.join(cur_dir, 'tfrecords')
tfrecord_train = 'simpsons_train.tfrecord'
tfrecord_val = 'simpsons_val.tfrecord'

train_tfr_dir = os.path.join(tfrecord_dir, tfrecord_train)
val_tfr_dir = os.path.join(tfrecord_dir, tfrecord_val)

In [ ]:
def _parse_function(tfrecord_serialized):
    features={'image': tf.io.FixedLenFeature([], tf.string),
             'label': tf.io.FixedLenFeature([], tf.int64)}
    parsed_features = tf.io.parse_single_example(tfrecord_serialized, features)
    
    image = tf.io.decode_raw(parsed_features['image'], tf.uint8)
    image = tf.reshape(image, [img_size, img_size, 3])
    image = tf.cast(image, tf.float32)/255.
    
    label = tf.cast(parsed_features['label'], tf.int32)
    label = tf.one_hot(label, depth=n_class)
    
    return image, label

In [ ]:
train_dataset = tf.data.TFRecordDataset(train_tfr_dir)
train_dataset = train_dataset.map(_parse_function, num_parallel_calls=8)
train_dataset = train_dataset.shuffle(buffer_size=n_train*2).prefetch(
    buffer_size=batch_size).batch(batch_size).repeat()

In [ ]:
val_dataset = tf.data.TFRecordDataset(val_tfr_dir)
val_dataset = val_dataset.map(_parse_function, num_parallel_calls=8)
val_dataset = val_dataset.prefetch(buffer_size=batch_size).batch(batch_size)

In [ ]:
model = models.Sequential()
model.add(Conv2D(32, 3, input_shape=(img_size, img_size, 3)))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Conv2D(64, 3))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling2D())
model.add(Conv2D(128, 3))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Conv2D(256, 3))
model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPooling2D())
model.add(Conv2D(512, 3))
model.add(BatchNormalization())
model.add(ReLU())
model.add(Conv2D(512, 3))
model.add(BatchNormalization())
model.add(ReLU())
model.add(GlobalAveragePooling2D())
model.add(Dense(n_class))
model.add(BatchNormalization())
model.add(Softmax())
model.summary()

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate,
                                                          decay_steps=n_train//batch_size*10,
                                                          decay_rate=0.5,
                                                          staircase=True)
model.compile(optimizers.Adam(lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
training_epochs=5
history = model.fit(train_dataset, epochs=training_epochs, steps_per_epoch=n_train//batch_size,
                   validation_data=val_dataset, validation_steps=n_val//batch_size)

In [ ]:
train_dir = os.path.join(cur_dir, 'simpsons_train')
class_names = sorted([dname for dname in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, dname))])
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img[:,:,:])

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array[i], true_label[i]
    plt.grid(False)
    #plt.xticks([])
    plt.xticks(range(n_class), class_names, rotation=90)
    plt.yticks([])
    thisplot = plt.bar(range(n_class), predictions_array, color="#777777")
    plt.ylim([0, 1]) 
    predicted_label = np.argmax(predictions_array)
    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

In [ ]:
rnd_idx = np.random.randint(1, n_val//batch_size)
img_cnt = 0
for images, labels in val_dataset:
    img_cnt += 1
    if img_cnt != rnd_idx:
        continue
    predictions = model(images, training=False)
    num_rows = 5
    num_cols = 3
    num_images = num_rows*num_cols
    labels = tf.argmax(labels, axis=-1)
    plt.figure(figsize=(3*2*num_cols, 4*num_rows))
    plt.subplots_adjust(hspace=1.0)
    for i in range(num_images):
        plt.subplot(num_rows, 2*num_cols, 2*i+1)
        plot_image(i, predictions.numpy(), labels.numpy(), images.numpy())
        plt.subplot(num_rows, 2*num_cols, 2*i+2)
        plot_value_array(i, predictions.numpy(), labels.numpy())        
    break